In [5]:
from utils import TTS

#os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [6]:
# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
model_name = "facebook/nllb-200-distilled-600M"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to translate German to English. 
# If prompt is in english then no translate is needed
# Output the result
from langdetect import detect
def translate_prompt(prompt):
  is_prompt_english = detect(prompt) == 'en'
  if not is_prompt_english:
    # translate German to English
    prompt_src_lang = 'als_Latn'
    prompt_targt_lang = 'eng_Latn'
    translator = pipeline('translation', model=model,
                          tokenizer=tokenizer, src_lang=prompt_src_lang, tgt_lang=prompt_targt_lang)
    output = translator(prompt, max_length=1024)
    print(output)
    prompt = output[0]['translation_text']
    print(prompt)

# Invoke the tranlsate-function and get results
first_prompt = "t. E ëma, Maria, ishte shtëpiake. Familja u shtua shumë dhe erdhi një kohë që jetesa u bë e vështirë për të. Sa qe i mitur Noli hoqi sëmundje të rënda, prandaj shkollën e nisi me vonesë. Filloren dhe të mesmen i bëri greqisht. Por me shumë ndikoi tek ai krenaria e fshatit shqiptar për të kaluarën historike dhe dashuria për shkrimin shqip.Libri i pare qe ka lexuar ne shqip ka qen Dhjata e Re me perkthim te Kristoforidhit."
second_prompt = "Beispiel 2: Ich bin ein Berliner."

translate_prompt(first_prompt)
#translate_prompt(second_prompt)

[{'translation_text': 'His mother, Maria, was a homemaker. The family grew very much and there came a time when life became difficult for him. As a young girl, Noly got rid of a serious illness, so he started school late.'}]
His mother, Maria, was a homemaker. The family grew very much and there came a time when life became difficult for him. As a young girl, Noly got rid of a serious illness, so he started school late.


In [7]:
tokenizer("How was your day?").input_ids

[256162, 13374, 1398, 4260, 4039, 248130, 2]

In [8]:
import torch 
torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4050 Laptop GPU'

In [9]:
im_cp = TTS()

INIT


In [10]:
#im_cp.subtitles_video_with_display('sample3.mp4',is_url=False,url='https://youtu.be/o-GdKXiM0gU',display=True,languag='en',save=False)

In [11]:
im_cp.translate = True
#im_cp.translator.set_source_language ('eng_Latn')
im_cp.translator.set_target_language ('ita_Latn')


In [12]:
im_cp.subtitles_video('sample4.mp4',chunk_length=-1,save_txt=True)

: 

'als_Latn'

In [ ]:
from utils import write_subs_to_video
write_subs_to_video(video_fn="sample_tr_dize.mp4",subs_fn="sample_tr_dize.srt",output_path='sample_en_translated.mp4',font_sz=20,max_length=60)

In [1]:
srtfile = open("/home/kriselda/Downloads/arrival.txt", "r")
ARRIVAL = srtfile.read()
srtfile.close()

In [3]:
with open("full_episode.txt", "r") as srtfile:
    NANNY = srtfile.read()


In [1]:
from utils import get_text_from_srt
NICEGUYS=get_text_from_srt('theniceguys.srt')
OTHER = get_text_from_srt('theotherguys.srt')
B99 = get_text_from_srt('b99.srt')
LO = get_text_from_srt('lawandorder.srt')

In [3]:
from utils import divide_text_into_sentence_batches
input_batches = divide_text_into_sentence_batches(input_text,max_tokens_per_batch=900)

In [6]:
#flan is not working for summarization
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
for input_text_n in input_batches:
    input_ids = tokenizer('Summarize : '+ input_text_n, return_tensors="pt").input_ids

    outputs = model.generate(input_ids)
    print(tokenizer.decode(outputs[0]))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Token indices sequence length is longer than the specified maximum sequence length for this model (1050 > 512). Running this sequence through the model will result in indexing errors


In [4]:
from transformers import BartTokenizer, BartForConditionalGeneration
# philschmid/bart-large-cnn-samsum is better that flan-t5 but not acceptable
# Load pre-trained BART tokenizer and model
tokenizer = BartTokenizer.from_pretrained('philschmid/flan-t5-base-samsum')
model = BartForConditionalGeneration.from_pretrained('philschmid/bart-large-cnn-samsum')

# Input text to be summarized
for input_text_n in input_batches:

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt', max_length=1024, truncation=True)

    # Generate the summary
    summary_ids = model.generate(input_ids, max_length=200, min_length=50, num_beams=4, early_stopping=True)

    # Decode the generated summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Print the generated summary
    print("Generated Summary:")
    print(summary)



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'BartTokenizer'.


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
model = GPT2LMHeadModel.from_pretrained("gpt2-large")

# Prompt for summarization
for prompt in input_batches:
    # Tokenize input
    input_ids = tokenizer.encode(prompt + '/n : summarize', return_tensors="pt", max_length=1024, truncation=True)

    # Generate output
    output = model.generate(input_ids, max_length=150, num_return_sequences=1, early_stopping=True)

    # Decode and print the summarized text
    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Summarized text:")
    print(summary)


In [6]:
from transformers import pipeline
from utils import divide_text_into_sentence_batches

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

batches = divide_text_into_sentence_batches(B99,max_tokens_per_batch=900)
for i in range(len(batches)):

    print(summarizer(batches[i], max_length=130, min_length=30, do_sample=False))


[{'summary_text': 'Pimento returns to the NYPD after spending time in prison in Uzbekistan. Pimento and Jake plan to get married in 14 hours. Jake asks Rafael UPD for a wedding.'}]
[{'summary_text': '"Empire" is about two best friends getting married in New York City. The show is set to air on Sunday night at 10 p.m. ET.'}]
[{'summary_text': "Jake and Rosa are trying to find his grandma's earrings. The store where the earrings were bought burned down. Jake decides to ignore the sign and marry Rosa anyway."}]
[{'summary_text': "Rosa is drunk off champers at a Renaissance Faire. She asks Charles to marry her, but he says he's not sober enough. Rosa calls the local police and asks them to get a warrant. Rosa and her friends rob a store and try to steal the owner's earrings. They find the earrings, then light the store on fire."}]
[{'summary_text': 'Adrian: "I\'m so sick of this garbage, all right? The two of you need to stop looking for signs everywhere... oh, my God, a sign. Whoo! He can

: 

In [3]:
len(OTHER[:20000].split(' '))

3400

In [33]:
NICEGUYS[50000:]

'ook.\n"Opening night, 9:00 p.m." Signed, Chet.\n- Fucking Chet. - The protestor guy?\nGive me that shit.\nShe was planning something with Chet.\nOpening night?\nThe LA auto show.\n- It\'s today, right? - Yeah.\nBig party. Mucky-mucks. Loads of press.\nIf you wanted to get a story out there, right?\nAnd fucking Chet\'s a projectionalist.\nPlease stop talking.\nI\'ve been listening to everything you said.\nDoes this mean... Does this mean that my niece is dead?\nYes!\nI mean, you know, yes.\n- She was murdered. I\'m sorry. - Mmm-hmm.\nBut we\'re gonna bring down the people who did it.\nYeah, and for a deeply discounted rate, so...\nWelcome to Los Angeles and the 1978 Pacific Coast Auto Show.\nStyled road wheels.\nThese all-new fuel-efficient systems.\nAn incredible 210-brake horsepower.\nTheir battery-powered three-passenger runabout.\nCadillac have brought their new El Dorado.\nFeaturing a 7-liter V8.\nYou guys know where the projection room is?\nYou seen Chet, the projectionist?\nYeah

In [23]:
len(B99)

22203

In [22]:
len(B99.split(' '))

3586